In [ ]:
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

# === Settings ===
API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
K = 16
NUM_SAMPLES_PER_DEMO = 5
LEARNING_RATE =  1e-5
MAX_STEPS = 5
TEMPERATURE = 0.7

# === Init ===
encoder = MathBERTEncoder(device=DEVICE, trainable=True)
encoder.train()

icl_model = OpenAIICLModel(api_key=API_KEY, model_name="gpt-4.1-nano", temperature=TEMPERATURE)
optimizer = torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)

gsm8k_data = load_dataset('gsm8k', 'main')['train']
gsm8k_data = gsm8k_data.select(range(256))  # slice first 200 examples
gsm8k_data_to_infer = gsm8k_data.select(range(5))  # slice first 200 examples

# === Training Loop ===
for step in tqdm(range(MAX_STEPS), desc="Training Steps"):
    print(f"\n=== Training Step {step+1} ===")

    for inference_index in tqdm(range(len(gsm8k_data_to_infer)), desc="Examples"):
        inference_item = gsm8k_data[inference_index]
        demo_pool = [d for idx, d in enumerate(gsm8k_data) if idx != inference_index]

        Q_inf = inference_item["question"]
        A_gt = inference_item["answer"]
        demos = [(d["question"], d["answer"]) for d in demo_pool]

        q_emb = encoder.encode([Q_inf], detach=False).squeeze(0)
        demo_embs = encoder.encode([q for (q, a) in demos], detach=False)

        top_k_indices, similarities = rc.retrieve_sample_k_cosine(q_emb, demo_embs, k=min(K, len(demos)))
        selected_demos = [demos[i] for i in top_k_indices]

        print(f"\n🧠 Inference Index {inference_index}")
        print(f"🔍 Top-K Indices: {top_k_indices}")

        all_responses = sample_responses_per_demo(
            demo_tuples=selected_demos,
            Q_inf=Q_inf,
            icl_model=icl_model,
            num_samples=NUM_SAMPLES_PER_DEMO
        )

        rewards = []
        for i, responses in enumerate(all_responses):
            reward = compute_demo_accuracy(responses, A_gt)
            rewards.append(reward)
            # print(f"    Demo {i} | Reward: {reward:.2f}")

        rewards = torch.tensor(rewards, dtype=torch.float32).to(DEVICE)

        loss = grpo_step(
            rewards,
            similarities,
            q_emb,
            demo_embs,
            optimizer
        )

        print(f"✅ Loss: {loss:.4f}")



Training Steps:   0%|          | 0/5 [00:00<?, ?it/s]


=== Training Step 1 ===



🧠 Inference Index 0
🔍 Top-K Indices: tensor([194, 131, 242, 157,  77, 146, 175,   4, 160, 238, 197, 222, 249, 199,
        220, 168], device='cuda:0')
tensor([ 0.8456,  0.8456, -0.5074,  0.8456, -0.9583,  0.8456,  0.8456,  0.8456,
         0.8456,  0.8456, -1.4093, -0.0564, -1.4093, -1.4093,  0.3946, -1.4093],
       device='cuda:0')
tensor(0.4435, device='cuda:0')
tensor([-5.1454, -4.3108, -8.3210, -5.2740, -4.4696, -5.1823, -5.6701, -4.7556,
        -3.6997, -5.5623, -5.4184, -4.9934, -5.0418, -6.1091, -4.2354, -5.9010],
       device='cuda:0', grad_fn=<IndexBackward0>)


✅ Loss: -0.3311

🧠 Inference Index 1
🔍 Top-K Indices: tensor([218,  87, 154, 163,  17, 226,  82,  67, 143, 197,  76,  33, 104, 220,
         84, 196], device='cuda:0')
tensor([-2.1137, -0.6433,  0.8271,  0.8271,  0.8271,  0.8271,  0.8271,  0.8271,
        -1.3785,  0.0919,  0.0919, -0.6433,  0.8271, -1.3785, -0.6433,  0.8271],
       device='cuda:0')
tensor(0.2720, device='cuda:0')
tensor([-5.2589, -5.8771, -5.4172, -4.5478, -6.0249, -5.9719, -6.0917, -4.8652,
        -6.2500, -4.3484, -4.1903, -5.8066, -5.7541, -5.0994, -4.8990, -5.8860],
       device='cuda:0', grad_fn=<IndexBackward0>)


✅ Loss: 0.0132

🧠 Inference Index 2
🔍 Top-K Indices: tensor([ 23,  16, 159,  64, 125, 185, 104,  22,  86, 138,  59, 236, 231,  11,
        132,   8], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
tensor(1.0000e-04, device='cuda:0')
tensor([-5.1143, -4.9787, -4.1707, -3.9710, -4.7787, -5.7246, -4.6564, -4.7204,
        -5.8670, -4.9582, -5.4938, -4.6734, -5.4807, -4.6598, -5.4189, -4.7008],
       device='cuda:0', grad_fn=<IndexBackward0>)


...

✅ Loss: -0.0000

🧠 Inference Index 3
🔍 Top-K Indices: tensor([ 10, 147,  88, 152, 157,   5, 194, 143,  61,  50,  11,  58,  60,  23,
        104, 129], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
tensor(1.0000e-04, device='cuda:0')
tensor([-3.4395, -4.6450, -6.1735, -6.0821, -4.3212, -3.9897, -5.7370, -5.1578,
        -6.7016, -5.5341, -5.4668, -5.4358, -5.4115, -5.0638, -4.6127, -4.2444],
       device='cuda:0', grad_fn=<IndexBackward0>)


✅ Loss: -0.0000

🧠 Inference Index 4
🔍 Top-K Indices: tensor([ 72, 198, 200, 124,  49,  45,  83,  11,  14, 222, 201,  34, 197,  92,
         31, 146], device='cuda:0')
tensor([-0.2500, -0.2500, -0.2500, -0.2500,  3.7500, -0.2500, -0.2500, -0.2500,
        -0.2500, -0.2500, -0.2500, -0.2500, -0.2500, -0.2500, -0.2500, -0.2500],
       device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor([-4.6053, -5.9042, -5.1781, -6.1039, -4.4286, -4.4074, -5.3460, -3.9077,
        -5.0142, -5.6397, -5.7098, -6.6044, -5.3218, -5.6438, -6.2366, -4.2818],
       device='cuda:0', grad_fn=<IndexBackward0>)


Training Steps: 100%|██████████| 5/5 [15:04<00:00, 181.00s/it]

✅ Loss: -0.2106


: 

In [ ]:
# Save the updated MathBERT model
save_path = "./updated_mathbert"  # your save directory
encoder.model.save_pretrained(save_path)
encoder.tokenizer.save_pretrained(save_path)

# LOADING

# from transformers import BertTokenizer, BertModel

# model = BertModel.from_pretrained("./updated_mathbert")
# tokenizer = BertTokenizer.from_pretrained("./updated_mathbert")
